In [ ]:
import argparse
import h5py
from sklearn.model_selection import train_test_split
import numpy as np

In [ ]:
file_dict = {
    'Background': '/tmp/all_data/background_for_training.h5',
    'Ato4l': '/tmp/all_data/Ato4l_lepFilter_13TeV_filtered.h5',
    'hChToTauNu': '/tmp/all_data/hChToTauNu_13TeV_PU20_filtered.h5',
    'hToTauTau' : '/tmp/all_data/hToTauTau_13TeV_PU20_filtered.h5', 
    'leptoquark': '/tmp/all_data/leptoquark_LOWMASS_lepFilter_13TeV_filtered.h5',
}

background_file = "/tmp/all_data/background_for_training.h5" # background data file

signal_files = [ # signal data files
    
    '/tmp/all_data/Ato4l_lepFilter_13TeV_filtered.h5',
    '/tmp/all_data/hChToTauNu_13TeV_PU20_filtered.h5',
    '/tmp/all_data/hToTauTau_13TeV_PU20_filtered.h5', 
    '/tmp/all_data/leptoquark_LOWMASS_lepFilter_13TeV_filtered.h5',
]

output_names = [ # names for output files
    "Ato4l_lepFilter",
    "hChToTauNu",
    "hToTauTau",
    "leptoquark_LOWMASS"
]

In [ ]:
def create_datasets_convolutional(bkg_file, output_bkg_name, signals_files, output_signal_names, events=None, test_size=0.2, val_size=0.2, input_shape=57):
    
    # read BACKGROUND data
    with h5py.File(bkg_file, 'r') as file:
        full_data = file['Particles'][:,:,:-1]
        np.random.shuffle(full_data)
        if events: full_data = full_data[:events,:,:]
    
    # define training, test and validation datasets
    X_train, X_test = train_test_split(full_data, test_size=test_size, shuffle=True)
    X_train, X_val = train_test_split(X_train, test_size=val_size)

    del full_data
    
    with h5py.File(output_bkg_name + '_dataset.h5', 'w') as h5f:
        h5f.create_dataset('X_train', data = X_train)
        h5f.create_dataset('X_test', data = X_test)
        h5f.create_dataset('X_val', data = X_val)
    
    if signals_files:
        # read SIGNAL data
        for i, signal_file in enumerate(signals_files):
            f = h5py.File(signal_file,'r')
            signal_data = f['Particles'][:,:,:-1]
            with h5py.File(output_signal_names[i] + '_dataset.h5', 'w') as h5f2:
                h5f2.create_dataset('Data', data = signal_data)
        
    return

In [ ]:
create_datasets_dense(background_file, )